# Writing out a USGSCSM ISD from a PDS3 Mex HRSC image

In [1]:
import os
# Update with where your metakernels are located
os.environ['ALESPICEROOT'] = '/scratch/spice/'
from ale.drivers.mex_drivers import MexHrscPds3NaifSpiceDriver
from ale.drivers.mex_drivers import MexHrscIsisLabelNaifSpiceDriver
from ale.formatters.isis_formatter import to_isis
import json
import ale

import os
from glob import glob

import numpy as np

import pvl
import struct
import spiceypy as spice
import warnings

from ale.base.data_isis import read_table_data
from ale.base.data_isis import parse_table
from ale.base import Driver
from ale.base.data_naif import NaifSpice
from ale.base.label_pds3 import Pds3Label
from ale.base.label_isis import IsisLabel
from ale.base.type_sensor import LineScanner
from ale.base.type_distortion import RadialDistortion
from ale.util import find_latest_metakernel

## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in a PDS3 image using NAIF SPICE kernels for exterior orientation data. If the driver utilizes NAIF SPICE kernels, it is implemented as a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers) and will furnish metakernels when entering the context (i.e. when entering the `with` block) and free the metakernels on exit. This maintains the integrity of spicelib's internal data structures. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions. 

The ALESPICEROOT environment variable is used to find metakernels for different missions. You can update the path in the notebook to point to your metakernels directory. The directory is required to follow the NAIF direcrtory setup.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `LroLrocPds3LabelNaifSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * A PDS3 Mex HRSC image
 * NAIF metakernels installed
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [2]:
fileName = '/home/tthatcher/Desktop/data/h5270_0000_ir2.cub'

isis_lbl = pvl.load(fileName)
kerns = pvl.dumps(isis_lbl["IsisCube"])
isis_lbl["IsisCube"]["Kernels"]

PVLGroup([
  ('NaifIkCode', -41218)
  ('LeapSecond', '$base/kernels/lsk/naif0012.tls')
  ('TargetAttitudeShape', '$base/kernels/pck/pck00009.tpc')
  ('TargetPosition',
   ['Table', '$base/kernels/spk/de430.bsp', '$base/kernels/spk/mar097.bsp'])
  ('InstrumentPointing',
   ['Table',
    '$mex/kernels/ck/ATNM_MEASURED_080101_090101_V03.BC',
    '$mex/kernels/fk/MEX_V14.TF'])
  ('Instrument', '$mex/kernels/ik/MEX_HRSC_V03.TI')
  ('SpacecraftClock', '$mex/kernels/sclk/MEX_190911_STEP.TSC')
  ('InstrumentPosition',
   ['Table', '$mex/kernels/spk/ORMM__080201000000_00474.BSP'])
  ('InstrumentAddendum', '$mex/kernels/iak/hrscAddendum004.ti')
  ('ShapeModel', '$base/dems/molaMarsPlanetaryRadius0005.cub')
  ('InstrumentPositionQuality', 'Reconstructed')
  ('InstrumentPointingQuality', 'Reconstructed')
  ('CameraVersion', 1)
])

In [3]:
isis_bytes = read_table_data(isis_lbl['Table'], fileName)
print(isis_bytes)
parse_table(isis_lbl['Table'], isis_bytes)

b'\x13\x12\x0e\xae\xb4|\xaeA\x19\x04V\x0eM7\x8a?\x01\x00\x00\x00\n\xf9\xa9X\xb5|\xaeAj\xbct\x938o\x8a?\t\x1a\x00\x00\xd0\xa9\xb0X\xb5|\xaeA\xac\x1cZd\xfb\x16\x8b?\n\x1a\x00\x00'


{'EphemerisTime': [255744599.02748165, 255744684.33197814, 255744684.34504557],
 'ExposureTime': [0.012800790786743165,
  0.012907449722290038,
  0.013227428436279297],
 'LineStart': [1, 6665, 6666],
 'Name': 'LineScanTimes',
 'StartByte': 77798913,
 'Bytes': 60,
 'Records': 3,
 'ByteOrder': 'Lsb'}

In [4]:
isis_lbl

PVLModule([
  ('IsisCube',
   {'Archive': {'DataSetId': 'MEX-M-HRSC-3-RDR-V2.0',
                'DetectorId': 'MEX_HRSC_IR',
                'EventType': 'MARS-REGIONAL-MAPPING-Vo-Te-Im',
                'OrbitNumber': 5270,
                'ProductId': 'H5270_0000_IR2.IMG'},
    'BandBin': {'Center': Units(value=955.5, units='nm'),
                'Width': Units(value=81.0, units='nm')},
    'Core': {'Dimensions': {'Bands': 1,
                            'Lines': 15088,
                            'Samples': 1288},
             'Format': 'Tile',
             'Pixels': {'Base': 0.0,
                        'ByteOrder': 'Lsb',
                        'Multiplier': 1.0,
                        'Type': 'Real'},
             'StartByte': 65537,
             'TileLines': 368,
             'TileSamples': 322},
    'Instrument': {'FocalPlaneTemperature': Units(value=7.9716, units='degC'),
                   'InstrumentId': 'HRSC',
                   'InstrumentTemperature': Units(value=11.03

In [5]:
from ale import util 
util.get_kernels_from_isis_pvl(isis_lbl["IsisCube"])


['$ISIS3DATA/base/kernels/spk/de430.bsp',
 '$ISIS3DATA/base/kernels/spk/mar097.bsp',
 '$ISIS3DATA/mex/kernels/spk/ORMM__080201000000_00474.BSP',
 '$ISIS3DATA/mex/kernels/ck/ATNM_MEASURED_080101_090101_V03.BC',
 '$ISIS3DATA/mex/kernels/fk/MEX_V14.TF',
 '$ISIS3DATA/base/kernels/pck/pck00009.tpc',
 '$ISIS3DATA/mex/kernels/ik/MEX_HRSC_V03.TI',
 '$ISIS3DATA/mex/kernels/iak/hrscAddendum004.ti',
 '$ISIS3DATA/base/kernels/lsk/naif0012.tls',
 '$ISIS3DATA/mex/kernels/sclk/MEX_190911_STEP.TSC']

In [6]:
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data/"

In [10]:
# change to desired PDS3 image path 
# fileName = '/home/kdlee/h5270_0000_ir2.img'
fileName = '/home/tthatcher/Desktop/data/h5270_0000_ir2.cub'

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 

with MexHrscIsisLabelNaifSpiceDriver(fileName, props={'kernels' : isis_lbl["IsisCube"]}) as driver:
    # pass driver instance into formatter function
    usgscsm_dict = to_isis(driver)
    print(usgscsm_dict)


{'CameraVersion': 1, 'NaifKeywords': {'BODY499_RADII': array([3396.19, 3396.19, 3376.2 ]), 'BODY_FRAME_CODE': 10014, 'BODY_CODE': 499, 'INS-41210_FOV_FRAME': 'MEX_HRSC_HEAD', 'FRAME_-41210_NAME': 'MEX_HRSC_HEAD', 'INS-41210_CK_TIME_TOLERANCE': 1.0, 'TKFRAME_-41210_AXES': array([1., 2., 3.]), 'TKFRAME_-41210_SPEC': 'ANGLES', 'FRAME_-41210_CLASS': 4.0, 'INS-41210_FOV_ANGULAR_SIZE': array([0.2     , 0.659734]), 'INS-41210_OD_K': array([0., 0., 0.]), 'INS-41210_F/RATIO': 5.6, 'INS-41210_PLATFORM_ID': -41000.0, 'TKFRAME_-41210_ANGLES': array([-0.334 ,  0.0101,  0.    ]), 'INS-41210_SPK_TIME_BIAS': 0.0, 'FRAME_-41210_CENTER': -41.0, 'TKFRAME_-41210_UNITS': 'DEGREES', 'INS-41210_BORESIGHT': array([  0.,   0., 175.]), 'INS-41210_CK_TIME_BIAS': 0.0, 'FRAME_-41210_CLASS_ID': -41210.0, 'INS-41210_IFOV': 4e-05, 'INS-41210_FOV_BOUNDARY_CORNERS': array([ 18.187 ,  60.0641, 175.    ,  18.1281, -60.0399, 175.    ,
       -18.1862, -60.0435, 175.    , -18.142 ]), 'INS-41210_FOV_SHAPE': 'RECTANGLE', 'TK

In [8]:
MexHrscIsisLabelNaifSpiceDriver.__mro__

(ale.drivers.mex_drivers.MexHrscIsisLabelNaifSpiceDriver,
 ale.base.type_sensor.LineScanner,
 ale.base.label_isis.IsisLabel,
 ale.base.data_naif.NaifSpice,
 ale.base.type_distortion.RadialDistortion,
 ale.base.base.Driver,
 object)

### Write ISD to disk 

ALE's USGSCSM formatter function returns a JSON. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.


In [9]:
usgscsm_dict.keys()

dict_keys(['CameraVersion', 'NaifKeywords', 'InstrumentPointing', 'BodyRotation', 'InstrumentPosition', 'SunPosition'])